# Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# importing our custom python file containing the code related to the model creation
import model

## Create Train and Test Dataloaders

In [2]:
def create_dataloaders(cuda_batch_size = 128, cpu_batch_size = 64):
  '''
  Creates train and test data loaders by applying transformations

  :param cuda_batch_size: batch size for cuda
  :param cpu_batch_size: batch size for cpu, if cuda is not available
  '''
  # Train Phase transformations
  train_transforms = transforms.Compose([
                                      transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.RandomAffine(degrees=7, shear=10, translate=(0.1, 0.1), scale=(0.8, 1.2)),
                                       transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.40, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) 
                                       ])

  # Test Phase transformations
  test_transforms = transforms.Compose([
                                      transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])



  train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
  test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

  SEED = 1

  # CUDA?
  cuda = torch.cuda.is_available()
  print("CUDA Available?", cuda)

  # For reproducibility
  torch.manual_seed(SEED)

  if cuda:
      torch.cuda.manual_seed(SEED)

  # dataloader arguments
  dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

  # train dataloader
  train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

  # test dataloader
  test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

  return train_loader, test_loader



In [3]:
# Let's now actually create the data loaders
train_loader, test_loader = create_dataloaders()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

CUDA Available? True


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# The Model and Params


In [4]:
!pip install torchsummary
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

dropout_rate = 0.05
no_of_groups_for_GN, no_of_groups_for_LN = 4, 1

# For next set of assignmnets we would move this code onto createmodel python file
# and would save the weights
def create_BN_GN_LN_models(device, dropout_rate, no_of_groups_for_GN, no_of_groups_for_LN):
  '''
  Creates models by referring the model file and returns

  :param device: cuda (gpu) or cpu
  :param dropout_rate: dropout rate
  :param no_of_groups_for_GN: number of groups for group normalization
  :param no_of_groups_for_LN: number of groups for layer normalization, typically 1
  '''
  model_BN = model.Net(norm_type = 'BN', dropout_value = dropout_rate).to(device)
  model_GN = model.Net(norm_type = 'GN', dropout_value = dropout_rate).to(device)
  model_LN = model.Net(norm_type = 'LN', dropout_value = dropout_rate).to(device)
  return model_BN, model_GN, model_LN

model_BN, model_GN, model_LN = create_BN_GN_LN_models(device, dropout_rate, no_of_groups_for_GN, no_of_groups_for_LN)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print('Batch Normalization Model Parameters: ')
summary(model_BN, input_size=(1, 28, 28))

Batch Normalization Model Parameters: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32


In [ ]:
print('Group Normalization Model Parameters: ')
summary(model_GN, input_size=(1, 28, 28))

Group Normalization Model Parameters: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
         GroupNorm-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
         GroupNorm-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
        GroupNorm-13           [-1, 16, 10, 10]              32


In [ ]:
print('Layer Normalization Model Parameters: ')
summary(model_LN, input_size=(1, 28, 28))

Layer Normalization Model Parameters: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
         GroupNorm-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
         GroupNorm-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
        GroupNorm-13           [-1, 16, 10, 10]              32


# Training and Testing

In [5]:
from tqdm import tqdm
from torch.optim.lr_scheduler import LambdaLR, StepLR, OneCycleLR
import numpy as np
import math
import pandas as pd
import pickle

model_metrics = {}
epochs = 20


def train(model, device, train_loader, optimizer, 
          epoch,lambda_l1,scheduler,model_key,
          train_accuracy_key,train_loss_key):
  '''
    This method is responsible for training the model.
    We are building the dictionary like the following - 
    {
      'Model Key': {
        'training_accuracy': [],
        'training_loss': [],
        'testing_accuracy': [],
        'testing_loss': []
      }
    }

    Model keys are kept in the following form - 
    Model_<normalization type>_<l1 factor>_<l2 factor>

    Normalization type can be BN, LN or GN. BN is used for batch
    normalization, LN for layer normalization, and GN for
    group normalization.

    :param model: model
    :param device: device, cuda (gpu) or cpu
    :param train_loader: train loader
    :param optimizer:  optimizer, adam, sgd, for example
    :param epoch: epoch, number of times we are seeing entire training data
    :param lambda_l1: l1 regularizon parameter
    :param scheduler: scheduler
    :param model_key: model key to identify model specific metrics
    :param train_accuracy_key: train accuracy key for that model
    :param train_loss_key: train loss key for that model
  '''

  train_loss_local, train_accuracy_local = [], []
  

  
  if model_key not in model_metrics.keys():
    model_metrics[model_key] = {}
  
  if train_accuracy_key not in model_metrics[model_key].keys():
    model_metrics[model_key][train_accuracy_key] = []
  
  if train_loss_key not in model_metrics[model_key].keys():
    model_metrics[model_key][train_loss_key] = []
  
  correct, processed = 0, 0

  model.train()
  pbar = tqdm(train_loader)
  enumeration_count = 0
  
  for batch_idx, (data, target) in enumerate(pbar):
    enumeration_count += 1
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    y_pred = model(data)
    loss = F.nll_loss(y_pred, target)
    
    #L1 Regularization
    
    if lambda_l1 > 0:
      l1 = 0
      for p in model.parameters():
        l1 = l1 + p.abs().sum()
      loss = loss + lambda_l1*l1
    

    train_loss_local.append(loss.data.cpu().numpy().item())

    # Backpropagation
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True) 
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_accuracy_local.append(100*correct/processed)

  model_metrics[model_key][train_loss_key].append(loss.item())
  model_metrics[model_key][train_accuracy_key].append(round(100*correct/processed, 2))

def test(model, device, test_loader,model_key,
          test_accuracy_key,test_loss_key):
  '''
  This method is responsible for testing the model.
  Alongwith that we are keeping track of misclassified 10 images per model.

  Like earlier we showed in training,
  we are maintaining the dictionary in the following form - 
  
    We are building the dictionary like the following - 
    {
      'Model Key': {
        'training_accuracy': [],
        'training_loss': [],
        'testing_accuracy': [],
        'testing_loss': []
      }
    }

    Model keys are kept in the following form - 
    Model_<normalization type>_<l1 factor>_<l2 factor>

    Normalization type can be BN, LN or GN. BN is used for batch
    normalization, LN for layer normalization, and GN for
    group normalization.

    :param model: model
    :param device: device, cuda (gpu) or cpu
    :param test_loader: test loader
    :param model_key: model key to identify model specific metrics
    :param test_accuracy_key: test accuracy key for that model
    :param test_loss_key: test loss key for that model
  '''
  wrong_images, wrong_label, correct_label = [], [], []
  current_misprediction_count = 0

  if model_key not in model_metrics.keys():
    model_metrics[model_key] = {}
  
  if test_accuracy_key not in model_metrics[model_key].keys():
    model_metrics[model_key][test_accuracy_key] = []
  
  if test_loss_key not in model_metrics[model_key].keys():
    model_metrics[model_key][test_loss_key] = []

  model.eval()
  test_loss, correct = 0, 0
  correct = 0
  with torch.no_grad():
      for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          test_loss += F.nll_loss(output, target, reduction='sum').item()
          pred = output.argmax(dim=1, keepdim=True)
          correct += pred.eq(target.view_as(pred)).sum().item()

          wrong_pred = (pred.eq(target.view_as(pred)) == False)

          if current_misprediction_count < 10:
            wrong_images.append(data[wrong_pred])
            wrong_label.append(pred[wrong_pred])
            correct_label.append(target.view_as(pred)[wrong_pred])  
            wrong_predictions = list(zip(torch.cat(wrong_images),torch.cat(wrong_label),torch.cat(correct_label))) 
            current_misprediction_count += 1

  test_loss /= len(test_loader.dataset)
  model_metrics[model_key][test_loss_key].append(test_loss)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
  model_metrics[model_key][test_accuracy_key].append(100. * correct / len(test_loader.dataset))
  return wrong_predictions

def get_models():
  '''
  This method maintains the dictionary as per model key.
  Per model key, we have the model, norm type, l1 factor and l2 factor, as dictionary values.
  '''
  dict_models = {}
  dict_models['Model_BN_0_0'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': 0, 'L2_Factor': 0}
  dict_models['Model_BN_0_001'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': 0, 'L2_Factor': .001}
  dict_models['Model_BN_0_002'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': 0, 'L2_Factor': .002}

  dict_models['Model_BN_001_0'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': .001, 'L2_Factor': 0}
  dict_models['Model_BN_001_001'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': .001, 'L2_Factor': .001}
  dict_models['Model_BN_001_002'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': .001, 'L2_Factor': .002}

  dict_models['Model_BN_002_0'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': .002, 'L2_Factor': 0}
  dict_models['Model_BN_002_001'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': .002, 'L2_Factor': .001}
  dict_models['Model_BN_002_002'] = {'Model': model_BN,'Norm_type': 'BN','L1_Factor': .002, 'L2_Factor': .002}

  dict_models['Model_LN_0_0'] = {'Model': model_LN,'Norm_type': 'LN','L1_Factor': 0, 'L2_Factor': 0}
  dict_models['Model_LN_0_001'] = {'Model': model_LN,'Norm_type': 'LN','L1_Factor': 0, 'L2_Factor': .001}
  dict_models['Model_LN_0_002'] = {'Model': model_LN,'Norm_type': 'LN','L1_Factor': 0, 'L2_Factor': .002}


  dict_models['Model_GN_0_0'] = {'Model': model_GN,'Norm_type': 'GN','L1_Factor': 0, 'L2_Factor': 0}
  dict_models['Model_GN_0_001'] = {'Model': model_GN,'Norm_type': 'GN','L1_Factor': 0, 'L2_Factor': .001}
  dict_models['Model_GN_0_002'] = {'Model': model_GN,'Norm_type': 'GN','L1_Factor': 0, 'L2_Factor': .002}

  return dict_models

def get_l2_factors():
  '''
  This method is returning the set of l2 factors we are using.
  '''
  return [0, .001, .002]

def get_l2_factor_model_name_extension(l2factor, l1factor, norm):
  '''
  This method is an utility method, it forms the model key
  from the factors and norm, and returns it.

  :param l2factor: l2 factor
  :param l1factor: l1 factor
  :param norm: normalization type, can be BN, LN or GN
  '''
  if l1factor == .001:
    l1factor_local = '001'
  elif l1factor == .002:
    l1factor_local = '002'
  else:
    l1factor_local = '0'
  
  if l2factor == .001:
    l2factor_local = '001'
  elif l2factor == .002:
    l2factor_local = '002'
  else:
    l2factor_local = '0'
  
  return 'Model_' + norm + '_' + l1factor_local + '_' + l2factor_local
  


def experiments(device, model_key):
  '''
  This method is used for doing experiment per each model.
  It calls train and test method to calculate and note accuracies and losses.

  :param device: device, cuda (gpu) or cpu
  :param model_key: model key
  '''
    dict_models = get_models()
    weight_decays = get_l2_factors()
    wrong_predictions = {}

    #for m in dict_models.keys():
      #for wd in weight_decays:
    m = model_key
    model_key_local = get_l2_factor_model_name_extension(dict_models[m]['L2_Factor'], 
                                                          dict_models[m]['L1_Factor'], dict_models[m]['Norm_type'])
    
    

    optimizer = optim.SGD(dict_models[m]['Model'].parameters(), lr=0.015, momentum=0.7,weight_decay= dict_models[m]['L2_Factor'])
    scheduler = OneCycleLR(optimizer, max_lr=0.015,epochs=epochs,steps_per_epoch=len(train_loader))
    wrong_predictions[model_key_local] = []

    for epoch in range(1, epochs + 1):
      print(f'Epoch {epoch}:')
      train(dict_models[m]['Model'], device, train_loader, optimizer, 
          epoch,dict_models[m]['L1_Factor'],scheduler, model_key_local,
          'Train_Accuracy','Train_Loss')
      wrong_predictions[model_key_local].append(test(dict_models[m]['Model'], 
                                                                device, test_loader, model_key_local,
          'Test_Accuracy', 'Test_Loss'))
    
    df_metrics = pd.DataFrame(model_metrics[model_key_local])
    df_metrics.to_csv(model_key_local + '.csv', index=False)

    with open(model_key_local + '.pkl', 'wb') as f:
      pickle.dump(wrong_predictions, f)
    


    
     

#Experiment Results - 

In [ ]:
print('Train and Test stats for BN, with L1 factor 0, and L2 factor 0:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_0_0')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0, and L2 factor 0:
----------------------------------------------------------------
Epoch 1:


Loss=0.38140541315078735 Batch_id=937 Accuracy=73.01: 100%|██████████| 938/938 [01:35<00:00,  9.77it/s]



Test set: Average loss: 0.1256, Accuracy: 9669/10000 (96.69%)

Epoch 2:


Loss=0.06730850040912628 Batch_id=937 Accuracy=93.75: 100%|██████████| 938/938 [01:29<00:00, 10.49it/s]



Test set: Average loss: 0.0648, Accuracy: 9795/10000 (97.95%)

Epoch 3:


...


Test set: Average loss: 0.0158, Accuracy: 9948/10000 (99.48%)

Epoch 19:


Loss=0.006705074105411768 Batch_id=937 Accuracy=98.36: 100%|██████████| 938/938 [01:26<00:00, 10.87it/s]



Test set: Average loss: 0.0169, Accuracy: 9945/10000 (99.45%)

Epoch 20:


Loss=0.004204779397696257 Batch_id=937 Accuracy=98.42: 100%|██████████| 938/938 [01:27<00:00, 10.76it/s]



Test set: Average loss: 0.0164, Accuracy: 9948/10000 (99.48%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0, and L2 factor 0.001:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_0_001')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0, and L2 factor 0.001:
----------------------------------------------------------------
Epoch 1:


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loss=0.49202385544776917 Batch_id=468 Accuracy=63.45: 100%|██████████| 469/469 [00:34<00:00, 13.48it/s]



Test set: Average loss: 0.2637, Accuracy: 9499/10000 (94.99%)

Epoch 2:


Loss=0.27910783886909485 Batch_id=468 Accuracy=91.79: 100%|██████████| 469/469 [00:34<00:00, 13.46it/s]



Test set: Average loss: 0.0662, Accuracy: 9845/10000 (98.45%)

Epoch 3:


...


Test set: Average loss: 0.0164, Accuracy: 9951/10000 (99.51%)

Epoch 19:


Loss=0.011540145613253117 Batch_id=468 Accuracy=98.45: 100%|██████████| 469/469 [00:35<00:00, 13.27it/s]



Test set: Average loss: 0.0165, Accuracy: 9951/10000 (99.51%)

Epoch 20:


Loss=0.028170542791485786 Batch_id=468 Accuracy=98.45: 100%|██████████| 469/469 [00:37<00:00, 12.65it/s]



Test set: Average loss: 0.0163, Accuracy: 9954/10000 (99.54%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0, and L2 factor 0.002:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_0_002')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0, and L2 factor 0.002:
----------------------------------------------------------------
Epoch 1:


Loss=0.010706108994781971 Batch_id=468 Accuracy=98.46: 100%|██████████| 469/469 [00:34<00:00, 13.61it/s]



Test set: Average loss: 0.0186, Accuracy: 9947/10000 (99.47%)

Epoch 2:


Loss=0.1344427466392517 Batch_id=468 Accuracy=98.26: 100%|██████████| 469/469 [00:34<00:00, 13.57it/s]



Test set: Average loss: 0.0213, Accuracy: 9939/10000 (99.39%)

Epoch 3:


...


Test set: Average loss: 0.0165, Accuracy: 9952/10000 (99.52%)

Epoch 19:


Loss=0.08697152137756348 Batch_id=468 Accuracy=98.63: 100%|██████████| 469/469 [00:34<00:00, 13.49it/s]



Test set: Average loss: 0.0163, Accuracy: 9957/10000 (99.57%)

Epoch 20:


Loss=0.01687847077846527 Batch_id=468 Accuracy=98.71: 100%|██████████| 469/469 [00:35<00:00, 13.14it/s]



Test set: Average loss: 0.0164, Accuracy: 9959/10000 (99.59%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0.001, and L2 factor 0:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_001_0')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0.001, and L2 factor 0:
----------------------------------------------------------------
Epoch 1:


Loss=0.48641911149024963 Batch_id=468 Accuracy=98.59: 100%|██████████| 469/469 [00:35<00:00, 13.13it/s]



Test set: Average loss: 0.0208, Accuracy: 9937/10000 (99.37%)

Epoch 2:


Loss=0.4028869867324829 Batch_id=468 Accuracy=98.03: 100%|██████████| 469/469 [00:35<00:00, 13.13it/s]



Test set: Average loss: 0.0282, Accuracy: 9916/10000 (99.16%)

Epoch 3:


...


Test set: Average loss: 0.0255, Accuracy: 9922/10000 (99.22%)

Epoch 19:


Loss=0.2460157871246338 Batch_id=468 Accuracy=97.91: 100%|██████████| 469/469 [00:36<00:00, 12.92it/s]



Test set: Average loss: 0.0209, Accuracy: 9934/10000 (99.34%)

Epoch 20:


Loss=0.24666491150856018 Batch_id=468 Accuracy=98.19: 100%|██████████| 469/469 [00:36<00:00, 12.79it/s]



Test set: Average loss: 0.0189, Accuracy: 9941/10000 (99.41%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0.001, and L2 factor 0.001:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_001_001')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0.001, and L2 factor 0.001:
----------------------------------------------------------------
Epoch 1:


Loss=0.2717297077178955 Batch_id=468 Accuracy=97.55: 100%|██████████| 469/469 [00:36<00:00, 12.88it/s]



Test set: Average loss: 0.0303, Accuracy: 9909/10000 (99.09%)

Epoch 2:


Loss=0.359923392534256 Batch_id=468 Accuracy=96.59: 100%|██████████| 469/469 [00:35<00:00, 13.15it/s]



Test set: Average loss: 0.0428, Accuracy: 9871/10000 (98.71%)

Epoch 3:


...


Test set: Average loss: 0.0258, Accuracy: 9932/10000 (99.32%)

Epoch 19:


Loss=0.2106304168701172 Batch_id=468 Accuracy=97.91: 100%|██████████| 469/469 [00:38<00:00, 12.30it/s]



Test set: Average loss: 0.0190, Accuracy: 9944/10000 (99.44%)

Epoch 20:


Loss=0.2342674732208252 Batch_id=468 Accuracy=98.18: 100%|██████████| 469/469 [00:36<00:00, 12.69it/s]



Test set: Average loss: 0.0182, Accuracy: 9945/10000 (99.45%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0.001, and L2 factor 0.002:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_001_002')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0.001, and L2 factor 0.002:
----------------------------------------------------------------
Epoch 1:


Loss=0.29474443197250366 Batch_id=468 Accuracy=97.56: 100%|██████████| 469/469 [00:36<00:00, 12.95it/s]



Test set: Average loss: 0.0326, Accuracy: 9902/10000 (99.02%)

Epoch 2:


Loss=0.3318052589893341 Batch_id=468 Accuracy=96.41: 100%|██████████| 469/469 [00:35<00:00, 13.06it/s]



Test set: Average loss: 0.0742, Accuracy: 9799/10000 (97.99%)

Epoch 3:


...


Test set: Average loss: 0.0248, Accuracy: 9924/10000 (99.24%)

Epoch 19:


Loss=0.2745257019996643 Batch_id=468 Accuracy=97.90: 100%|██████████| 469/469 [00:35<00:00, 13.13it/s]



Test set: Average loss: 0.0230, Accuracy: 9939/10000 (99.39%)

Epoch 20:


Loss=0.20330975949764252 Batch_id=468 Accuracy=98.15: 100%|██████████| 469/469 [00:37<00:00, 12.56it/s]



Test set: Average loss: 0.0205, Accuracy: 9941/10000 (99.41%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0.002, and L2 factor 0:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_002_0')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0.002, and L2 factor 0:
----------------------------------------------------------------
Epoch 1:


Loss=0.3975898325443268 Batch_id=468 Accuracy=97.35: 100%|██████████| 469/469 [00:35<00:00, 13.10it/s]



Test set: Average loss: 0.0580, Accuracy: 9836/10000 (98.36%)

Epoch 2:


Loss=0.5602798461914062 Batch_id=468 Accuracy=95.67: 100%|██████████| 469/469 [00:36<00:00, 12.85it/s]



Test set: Average loss: 0.0961, Accuracy: 9718/10000 (97.18%)

Epoch 3:


...


Test set: Average loss: 0.0286, Accuracy: 9917/10000 (99.17%)

Epoch 19:


Loss=0.3418366312980652 Batch_id=468 Accuracy=97.23: 100%|██████████| 469/469 [00:35<00:00, 13.03it/s]



Test set: Average loss: 0.0243, Accuracy: 9926/10000 (99.26%)

Epoch 20:


Loss=0.29006046056747437 Batch_id=468 Accuracy=97.67: 100%|██████████| 469/469 [00:36<00:00, 12.92it/s]



Test set: Average loss: 0.0235, Accuracy: 9924/10000 (99.24%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0.002, and L2 factor 0.001:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_002_001')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0.002, and L2 factor 0.001:
----------------------------------------------------------------
Epoch 1:


Loss=0.9140937328338623 Batch_id=937 Accuracy=75.10: 100%|██████████| 938/938 [01:21<00:00, 11.46it/s]



Test set: Average loss: 0.1347, Accuracy: 9686/10000 (96.86%)

Epoch 2:


Loss=0.8479819297790527 Batch_id=937 Accuracy=92.88: 100%|██████████| 938/938 [01:19<00:00, 11.73it/s]



Test set: Average loss: 0.0853, Accuracy: 9762/10000 (97.62%)

Epoch 3:


...


Test set: Average loss: 0.0430, Accuracy: 9873/10000 (98.73%)

Epoch 19:


Loss=0.3624892830848694 Batch_id=937 Accuracy=96.47: 100%|██████████| 938/938 [01:20<00:00, 11.72it/s]



Test set: Average loss: 0.0302, Accuracy: 9921/10000 (99.21%)

Epoch 20:


Loss=0.3535093069076538 Batch_id=937 Accuracy=97.03: 100%|██████████| 938/938 [01:19<00:00, 11.81it/s]



Test set: Average loss: 0.0296, Accuracy: 9920/10000 (99.20%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for BN, with L1 factor 0.002, and L2 factor 0.002:')
print('----------------------------------------------------------------')
experiments(device, 'Model_BN_002_002')
print('----------------------------------------------------------------')

Train and Test stats for BN, with L1 factor 0.002, and L2 factor 0.002:
----------------------------------------------------------------
Epoch 1:


Loss=0.387850284576416 Batch_id=937 Accuracy=95.09: 100%|██████████| 938/938 [01:22<00:00, 11.40it/s]



Test set: Average loss: 0.1014, Accuracy: 9671/10000 (96.71%)

Epoch 2:


Loss=0.49087855219841003 Batch_id=937 Accuracy=93.83: 100%|██████████| 938/938 [01:20<00:00, 11.67it/s]



Test set: Average loss: 0.0732, Accuracy: 9786/10000 (97.86%)

Epoch 3:


...


Test set: Average loss: 0.0575, Accuracy: 9840/10000 (98.40%)

Epoch 19:


Loss=0.3844752013683319 Batch_id=937 Accuracy=96.40: 100%|██████████| 938/938 [01:20<00:00, 11.66it/s]



Test set: Average loss: 0.0350, Accuracy: 9903/10000 (99.03%)

Epoch 20:


Loss=0.3983989357948303 Batch_id=937 Accuracy=97.05: 100%|██████████| 938/938 [01:19<00:00, 11.81it/s]



Test set: Average loss: 0.0319, Accuracy: 9910/10000 (99.10%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for LN, with L1 factor 0, and L2 factor 0:')
print('----------------------------------------------------------------')
experiments(device, 'Model_LN_0_0')
print('----------------------------------------------------------------')

Train and Test stats for LN, with L1 factor 0, and L2 factor 0:
----------------------------------------------------------------
Epoch 1:


Loss=0.17670778930187225 Batch_id=937 Accuracy=72.72: 100%|██████████| 938/938 [01:16<00:00, 12.25it/s]



Test set: Average loss: 0.1559, Accuracy: 9642/10000 (96.42%)

Epoch 2:


Loss=0.18419621884822845 Batch_id=937 Accuracy=94.03: 100%|██████████| 938/938 [01:17<00:00, 12.16it/s]



Test set: Average loss: 0.0643, Accuracy: 9812/10000 (98.12%)

Epoch 3:


...


Test set: Average loss: 0.0206, Accuracy: 9935/10000 (99.35%)

Epoch 19:


Loss=0.002684277016669512 Batch_id=937 Accuracy=98.75: 100%|██████████| 938/938 [01:15<00:00, 12.39it/s]



Test set: Average loss: 0.0185, Accuracy: 9935/10000 (99.35%)

Epoch 20:


Loss=0.011199524626135826 Batch_id=937 Accuracy=98.72: 100%|██████████| 938/938 [01:17<00:00, 12.16it/s]



Test set: Average loss: 0.0188, Accuracy: 9934/10000 (99.34%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for LN, with L1 factor 0, and L2 factor 0.001:')
print('----------------------------------------------------------------')
experiments(device, 'Model_LN_0_001')
print('----------------------------------------------------------------')

Train and Test stats for LN, with L1 factor 0, and L2 factor 0.001:
----------------------------------------------------------------
Epoch 1:


Loss=0.009237644262611866 Batch_id=937 Accuracy=98.68: 100%|██████████| 938/938 [01:17<00:00, 12.11it/s]



Test set: Average loss: 0.0200, Accuracy: 9932/10000 (99.32%)

Epoch 2:


Loss=0.02826235629618168 Batch_id=937 Accuracy=98.64: 100%|██████████| 938/938 [01:16<00:00, 12.32it/s]



Test set: Average loss: 0.0194, Accuracy: 9938/10000 (99.38%)

Epoch 3:


...


Test set: Average loss: 0.0177, Accuracy: 9952/10000 (99.52%)

Epoch 19:


Loss=0.18615347146987915 Batch_id=937 Accuracy=98.85: 100%|██████████| 938/938 [01:16<00:00, 12.33it/s]



Test set: Average loss: 0.0185, Accuracy: 9946/10000 (99.46%)

Epoch 20:


Loss=0.017266809940338135 Batch_id=937 Accuracy=98.82: 100%|██████████| 938/938 [01:18<00:00, 11.98it/s]



Test set: Average loss: 0.0175, Accuracy: 9949/10000 (99.49%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for LN, with L1 factor 0, and L2 factor 0.002:')
print('----------------------------------------------------------------')
experiments(device, 'Model_LN_0_002')
print('----------------------------------------------------------------')

Train and Test stats for LN, with L1 factor 0, and L2 factor 0.002:
----------------------------------------------------------------
Epoch 1:


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loss=0.4857370853424072 Batch_id=468 Accuracy=60.37: 100%|██████████| 469/469 [00:45<00:00, 10.41it/s]



Test set: Average loss: 0.3121, Accuracy: 9431/10000 (94.31%)

Epoch 2:


Loss=0.2110949605703354 Batch_id=468 Accuracy=91.53: 100%|██████████| 469/469 [00:38<00:00, 12.17it/s]



Test set: Average loss: 0.0888, Accuracy: 9788/10000 (97.88%)

Epoch 3:


...


Test set: Average loss: 0.0227, Accuracy: 9946/10000 (99.46%)

Epoch 19:


Loss=0.0874670147895813 Batch_id=468 Accuracy=98.69: 100%|██████████| 469/469 [00:37<00:00, 12.45it/s]



Test set: Average loss: 0.0221, Accuracy: 9945/10000 (99.45%)

Epoch 20:


Loss=0.02747148461639881 Batch_id=468 Accuracy=98.78: 100%|██████████| 469/469 [00:37<00:00, 12.50it/s]



Test set: Average loss: 0.0209, Accuracy: 9946/10000 (99.46%)

----------------------------------------------------------------


In [ ]:
print('Train and Test stats for GN, with L1 factor 0, and L2 factor 0:')
print('----------------------------------------------------------------')
experiments(device, 'Model_GN_0_0')
print('----------------------------------------------------------------')

Train and Test stats for GN, with L1 factor 0, and L2 factor 0:
----------------------------------------------------------------
Epoch 1:


Loss=0.4919760227203369 Batch_id=468 Accuracy=61.16: 100%|██████████| 469/469 [00:37<00:00, 12.62it/s]



Test set: Average loss: 0.2825, Accuracy: 9465/10000 (94.65%)

Epoch 2:


Loss=0.3325135409832001 Batch_id=468 Accuracy=91.53: 100%|██████████| 469/469 [00:37<00:00, 12.38it/s]



Test set: Average loss: 0.0841, Accuracy: 9795/10000 (97.95%)

Epoch 3:


...


Test set: Average loss: 0.0181, Accuracy: 9950/10000 (99.50%)

Epoch 19:


Loss=0.024716945365071297 Batch_id=468 Accuracy=98.44: 100%|██████████| 469/469 [00:38<00:00, 12.30it/s]



Test set: Average loss: 0.0173, Accuracy: 9953/10000 (99.53%)

Epoch 20:


Loss=0.04863129183650017 Batch_id=468 Accuracy=98.50: 100%|██████████| 469/469 [00:36<00:00, 12.79it/s]



Test set: Average loss: 0.0178, Accuracy: 9952/10000 (99.52%)

----------------------------------------------------------------


In [7]:
print('Train and Test stats for GN, with L1 factor 0, and L2 factor 0.001:')
print('----------------------------------------------------------------')
experiments(device, 'Model_GN_0_001')
print('----------------------------------------------------------------')

Train and Test stats for GN, with L1 factor 0, and L2 factor 0.001:
----------------------------------------------------------------
Epoch 1:


Loss=0.42129310965538025 Batch_id=468 Accuracy=68.45: 100%|██████████| 469/469 [00:37<00:00, 12.49it/s]



Test set: Average loss: 0.2518, Accuracy: 9531/10000 (95.31%)

Epoch 2:


Loss=0.13112914562225342 Batch_id=468 Accuracy=91.79: 100%|██████████| 469/469 [00:37<00:00, 12.61it/s]



Test set: Average loss: 0.0871, Accuracy: 9798/10000 (97.98%)

Epoch 3:


...


Test set: Average loss: 0.0208, Accuracy: 9943/10000 (99.43%)

Epoch 19:


Loss=0.07833284884691238 Batch_id=468 Accuracy=98.56: 100%|██████████| 469/469 [00:37<00:00, 12.63it/s]



Test set: Average loss: 0.0201, Accuracy: 9940/10000 (99.40%)

Epoch 20:


Loss=0.03007335029542446 Batch_id=468 Accuracy=98.61: 100%|██████████| 469/469 [00:38<00:00, 12.28it/s]



Test set: Average loss: 0.0199, Accuracy: 9941/10000 (99.41%)

----------------------------------------------------------------


In [8]:
print('Train and Test stats for GN, with L1 factor 0, and L2 factor 0.002:')
print('----------------------------------------------------------------')
experiments(device, 'Model_GN_0_002')
print('----------------------------------------------------------------')

Train and Test stats for GN, with L1 factor 0, and L2 factor 0.002:
----------------------------------------------------------------
Epoch 1:


Loss=0.03551127016544342 Batch_id=468 Accuracy=98.54: 100%|██████████| 469/469 [00:37<00:00, 12.56it/s]



Test set: Average loss: 0.0201, Accuracy: 9934/10000 (99.34%)

Epoch 2:


Loss=0.0522972047328949 Batch_id=468 Accuracy=98.32: 100%|██████████| 469/469 [00:36<00:00, 12.96it/s]



Test set: Average loss: 0.0252, Accuracy: 9923/10000 (99.23%)

Epoch 3:


...


Test set: Average loss: 0.0253, Accuracy: 9932/10000 (99.32%)

Epoch 19:


Loss=0.028558090329170227 Batch_id=468 Accuracy=98.69: 100%|██████████| 469/469 [00:37<00:00, 12.58it/s]



Test set: Average loss: 0.0211, Accuracy: 9940/10000 (99.40%)

Epoch 20:


Loss=0.03703347221016884 Batch_id=468 Accuracy=98.72: 100%|██████████| 469/469 [00:37<00:00, 12.50it/s]



Test set: Average loss: 0.0206, Accuracy: 9943/10000 (99.43%)

----------------------------------------------------------------
